In [1]:
import camelot
import pandas as pd

# --- helper function to identify transaction tables ---
def is_transaction_table(df):
    # Normalize column names
    # cols = [c.strip().lower() for c in df.columns]
    cols=[]
    for col in df.columns:
        cols.append(col.strip().lower())

    # Check if it matches the expected column pattern
    required_cols = {"date", "description", "amount", "balance"}
    if not required_cols.issubset(set(cols)):
        return False

    # Check for savings account keywords (to exclude)
    savings_keywords = ["international incoming wire fee", "book transfer credit b/o"]
    desc_text = " ".join(df.astype(str).apply(lambda x: " ".join(x), axis=1)).lower()

    if any(keyword in desc_text for keyword in savings_keywords):
        return False  # Skip savings account table

    return True


# --- main extraction function ---
def extract_transaction_tables(pdf_path):
    tables = camelot.read_pdf(pdf_path, pages="all", flavor="stream")
    valid_tables = []

    for i, table in enumerate(tables):
        df = table.df.copy()

        # Try to detect headers
        df.columns = [c.strip().lower() for c in df.iloc[0]]
        df = df[1:]  # remove the header row if duplicated

        if is_transaction_table(df):
            print(f"- Table {i} seems to be a checking transaction table.")
            valid_tables.append(df)
        else:
            print(f"* Table {i} skipped.")

    if valid_tables:
        final_df = pd.concat(valid_tables, ignore_index=True)
        print("\n Final combined transaction table:")
        print(final_df.head())
        return final_df
    else:
        print(" No checking transaction tables found.")
        return None


# --- call it here ---
pdf_path = r"D:\vidisha\vidi_documents\oct_bank_statement.pdf"   # 🔹 your file path
final_df = extract_transaction_tables(pdf_path)

* Table 0 skipped.
* Table 1 skipped.
- Table 2 seems to be a checking transaction table.
* Table 3 skipped.
* Table 4 skipped.
* Table 5 skipped.
* Table 6 skipped.
* Table 7 skipped.
* Table 8 skipped.

 Final combined transaction table:
    date                                        description  amount  balance
0                                         Beginning Balance          $192.67
1  09/26  Card Purchase           09/25 Weee Inc. 510-57...  -44.89   147.78
2  09/29  Card Purchase With Pin  09/27 Burlington Store...   -8.37   139.41
3                                                      5834                 
4  09/29  Card Purchase With Pin  09/27 Trader Joe S #55...  -10.96   128.45


In [2]:
final_df

date  \
0                             
1                     09/26   
2                     09/29   
3                             
4                     09/29   
5                             
6                     09/29   
7                     10/01   
8                     10/01   
9                     10/06   
10                    10/07   
11                    10/07   
12                    10/09   
13                    10/20   
14                    10/27   
15                    10/27   
16                            
17                    10/27   
18                            
19                    10/27   
20                    10/27   
21                            
22  *end*transaction detail   

                                          description     amount   balance  
0                                   Beginning Balance              $192.67  
1   Card Purchase           09/25 Weee Inc. 510-57...     -44.89    147.78  
2   Card Purchase With Pin  09/27 Burlington Store...      -8.37    139.41  
3                                                5834                       
4   Card Purchase With Pin  09/27 Trader Joe S #55...     -10.96    128.45  
5                                                5834                       
6   Card Purchase With Pin  09/27 Aldi 73013 Lake ...      -6.42    122.03  
7   Zelle Payment From Friend A
8   Online Transfer To  Sav ...XXXX Transaction#: ...     -25.00    124.48  
9   Card Purchase With Pin  10/05 Wal-Mart #2286 C...      -5.36    119.12  
10  Online Transfer From Sav ...XXXX Transaction#:...   1,191.00  1,310.12  
11  Zelle Payment From Friend A
12  Sbufeesdeposit   Purchase   168304          We...  -1,191.00    121.87  
13  Zelle Payment To Friend A Sbu 266477...     -10.25    111.62  
14  Online Transfer From Sav ...XXXX Transaction#:...      50.00    161.62  
15  Card Purchase           10/24 Sbu Sac Dunkin D...      -4.45    157.17  
16                                               5834                       
17  Card Purchase With Pin  10/25 Burlington Store...      -7.32    149.85  
18                                               5834                       
19  Card Purchase With Pin  10/25 Aldi 73013 Lake ...     -17.24    132.61  
20  Card Purchase With Pin  10/26 Wal-Mart #2286 C...     -13.53    119.08  
21                                     Ending Balance              $119.08  
22

In [4]:
current_balance = 119.08

for i in range(23,44):
    amount = final_df.loc[i, "amount"]

    # skip rows with no amount (like Beginning/Ending balance, placeholders, etc)
    if pd.isna(amount):
        continue
    
    # update balance one row at a time
    current_balance += amount
    final_df.loc[i, "balance"] = current_balance
final_df

date  \
0                             
1                     09/26   
2                     09/29   
3                             
4                     09/29   
5                             
6                     09/29   
7                     10/01   
8                     10/01   
9                     10/06   
10                    10/07   
11                    10/07   
12                    10/09   
13                    10/20   
14                    10/27   
15                    10/27   
16                            
17                    10/27   
18                            
19                    10/27   
20                    10/27   
21                            
22  *end*transaction detail   
23                    10/28   
24                    10/29   
25                    10/30   
26                    10/31   
27                    10/31   
28                    10/31   
29                    11/09   
30                    11/09   
31                    11/12   
32                    11/12   
33                    11/12   
34                    11/15   
35                    11/15   
36                    11/15   
37                    11/15   
38                    11/15   
39                    11/15   
40                    11/15   
41                    11/15   
42                    11/15   
43                    11/15   

                                          description     amount   balance  
0                                   Beginning Balance              $192.67  
1   Card Purchase           09/25 Weee Inc. 510-57...     -44.89    147.78  
2   Card Purchase With Pin  09/27 Burlington Store...      -8.37    139.41  
3                                                5834                       
4   Card Purchase With Pin  09/27 Trader Joe S #55...     -10.96    128.45  
5                                                5834                       
6   Card Purchase With Pin  09/27 Aldi 73013 Lake ...      -6.42    122.03  
7   Zelle Payment From Friend A
8   Online Transfer To  Sav ...XXXX Transaction#: ...     -25.00    124.48  
9   Card Purchase With Pin  10/05 Wal-Mart #2286 C...      -5.36    119.12  
10  Online Transfer From Sav ...XXXX Transaction#:...   1,191.00  1,310.12  
11  Zelle Payment From Friend A
12  Sbufeesdeposit   Purchase   168304          We...  -1,191.00    121.87  
13  Zelle Payment To Friend A Sbu 266477...     -10.25    111.62  
14  Online Transfer From Sav ...XXXX Transaction#:...      50.00    161.62  
15  Card Purchase           10/24 Sbu Sac Dunkin D...      -4.45    157.17  
16                                               5834                       
17  Card Purchase With Pin  10/25 Burlington Store...      -7.32    149.85  
18                                               5834                       
19  Card Purchase With Pin  10/25 Aldi 73013 Lake ...     -17.24    132.61  
20  Card Purchase With Pin  10/26 Wal-Mart #2286 C...     -13.53    119.08  
21                                     Ending Balance              $119.08  
22                                                                          
23  Online transfer from Sav ...XXXX Transaction#:...       50.0    169.08  
24                          Weee Inc. 510-573-4967 CA     -70.61     98.47  
25                        SBU-SAC Cafe Stony Brook NY     -12.89     85.58  
26  Online transfer from Sav ...XXXX Transaction#:...       26.7    112.28  
27  Online transfer from Student A ...      14.21    126.49  
28  Online transfer from Sav ...XXXX Transaction#:...       40.0    166.49  
29         SBU Sac Dunkin Donuts Stony Brook NY 11/09      -2.93    163.56  
30                           Aldi 73013 Lake Grove NY     -26.77    136.79  
31                Fedex32928309 800-463-3339 TN 11/13     -31.19     105.6  
32           Amazon.com*B86X05ZQ0 Amzn.com/bill 11/11     -31.19     74.41  
33           Amazon.com*BT3R00QI2 Amzn.com/bill 11/11       -5.4     69.01  
34  Online transfer from Sav ...XXXX Transactio

In [5]:
final_df['Web ID'] = final_df['description'].str.extract(r'Web ID[:#]?\s*(\d{10}$)')
final_df

date  \
0                             
1                     09/26   
2                     09/29   
3                             
4                     09/29   
5                             
6                     09/29   
7                     10/01   
8                     10/01   
9                     10/06   
10                    10/07   
11                    10/07   
12                    10/09   
13                    10/20   
14                    10/27   
15                    10/27   
16                            
17                    10/27   
18                            
19                    10/27   
20                    10/27   
21                            
22  *end*transaction detail   
23                    10/28   
24                    10/29   
25                    10/30   
26                    10/31   
27                    10/31   
28                    10/31   
29                    11/09   
30                    11/09   
31                    11/12   
32                    11/12   
33                    11/12   
34                    11/15   
35                    11/15   
36                    11/15   
37                    11/15   
38                    11/15   
39                    11/15   
40                    11/15   
41                    11/15   
42                    11/15   
43                    11/15   

                                          description     amount   balance  \
0                                   Beginning Balance              $192.67   
1   Card Purchase           09/25 Weee Inc. 510-57...     -44.89    147.78   
2   Card Purchase With Pin  09/27 Burlington Store...      -8.37    139.41   
3                                                5834                        
4   Card Purchase With Pin  09/27 Trader Joe S #55...     -10.96    128.45   
5                                                5834                        
6   Card Purchase With Pin  09/27 Aldi 73013 Lake ...      -6.42    122.03   
7   Zelle Payment From Friend A
8   Online Transfer To  Sav ...XXXX Transaction#: ...     -25.00    124.48   
9   Card Purchase With Pin  10/05 Wal-Mart #2286 C...      -5.36    119.12   
10  Online Transfer From Sav ...XXXX Transaction#:...   1,191.00  1,310.12   
11  Zelle Payment From Friend A
12  Sbufeesdeposit   Purchase   168304          We...  -1,191.00    121.87   
13  Zelle Payment To Friend A Sbu 266477...     -10.25    111.62   
14  Online Transfer From Sav ...XXXX Transaction#:...      50.00    161.62   
15  Card Purchase           10/24 Sbu Sac Dunkin D...      -4.45    157.17   
16                                               5834                        
17  Card Purchase With Pin  10/25 Burlington Store...      -7.32    149.85   
18                                               5834                        
19  Card Purchase With Pin  10/25 Aldi 73013 Lake ...     -17.24    132.61   
20  Card Purchase With Pin  10/26 Wal-Mart #2286 C...     -13.53    119.08   
21                                     Ending Balance              $119.08   
22                                                                           
23  Online transfer from Sav ...XXXX Transaction#:...       50.0    169.08   
24                          Weee Inc. 510-573-4967 CA     -70.61     98.47   
25                        SBU-SAC Cafe Stony Brook NY     -12.89     85.58   
26  Online transfer from Sav ...XXXX Transaction#:...       26.7    112.28   
27  Online transfer from Student A ...      14.21    126.49   
28  Online transfer from Sav ...XXXX Transaction#:...       40.0    166.49   
29         SBU Sac Dunkin Donuts Stony Brook NY 11/09      -2.93    163.56   
30                           Aldi 73013 Lake Grove NY     -26.77    136.79   
31                Fedex32928309 800-463-3339 TN 11/13     -31.19     105.6   
32           Amazon.com*B86X05ZQ0 Amzn.com/bill 11/11     -31.19     74.41   
33           Amazon.com*BT3R00QI2 Amzn.com/bill 11/11       -5.4     69.01   
34  Online tra

In [6]:
final_df['description'].str.extract(r'Transaction#[:]?\s*(\d{11}$)')    #this captures only those rows wich have the text:transaction# at the beginning,
                                                                        #doesnt capture the zelle payment rows which dont have this text, 
                                                                        #but do have a 11 digit transaction id at the end

#So we do this:
final_df['Transaction ID'] = final_df['description'].str.extract(r'(?:Transaction#[:]?\s*)?(\d{11}$)')

#Here we want to mark the entire grp Transaction#[:] as optional, not just the :
#so we wrap the entire grp in () and put a ? at the end
#also, we dont want to capture the optional text Transaction#[:], we just want to look for it wherever its present
#so we pu t ?: at the beginning of the optional text grp we're searching for

final_df

date  \
0                             
1                     09/26   
2                     09/29   
3                             
4                     09/29   
5                             
6                     09/29   
7                     10/01   
8                     10/01   
9                     10/06   
10                    10/07   
11                    10/07   
12                    10/09   
13                    10/20   
14                    10/27   
15                    10/27   
16                            
17                    10/27   
18                            
19                    10/27   
20                    10/27   
21                            
22  *end*transaction detail   
23                    10/28   
24                    10/29   
25                    10/30   
26                    10/31   
27                    10/31   
28                    10/31   
29                    11/09   
30                    11/09   
31                    11/12   
32                    11/12   
33                    11/12   
34                    11/15   
35                    11/15   
36                    11/15   
37                    11/15   
38                    11/15   
39                    11/15   
40                    11/15   
41                    11/15   
42                    11/15   
43                    11/15   

                                          description     amount   balance  \
0                                   Beginning Balance              $192.67   
1   Card Purchase           09/25 Weee Inc. 510-57...     -44.89    147.78   
2   Card Purchase With Pin  09/27 Burlington Store...      -8.37    139.41   
3                                                5834                        
4   Card Purchase With Pin  09/27 Trader Joe S #55...     -10.96    128.45   
5                                                5834                        
6   Card Purchase With Pin  09/27 Aldi 73013 Lake ...      -6.42    122.03   
7   Zelle Payment From Friend A
8   Online Transfer To  Sav ...XXXX Transaction#: ...     -25.00    124.48   
9   Card Purchase With Pin  10/05 Wal-Mart #2286 C...      -5.36    119.12   
10  Online Transfer From Sav ...XXXX Transaction#:...   1,191.00  1,310.12   
11  Zelle Payment From Friend A
12  Sbufeesdeposit   Purchase   168304          We...  -1,191.00    121.87   
13  Zelle Payment To Friend A Sbu 266477...     -10.25    111.62   
14  Online Transfer From Sav ...XXXX Transaction#:...      50.00    161.62   
15  Card Purchase           10/24 Sbu Sac Dunkin D...      -4.45    157.17   
16                                               5834                        
17  Card Purchase With Pin  10/25 Burlington Store...      -7.32    149.85   
18                                               5834                        
19  Card Purchase With Pin  10/25 Aldi 73013 Lake ...     -17.24    132.61   
20  Card Purchase With Pin  10/26 Wal-Mart #2286 C...     -13.53    119.08   
21                                     Ending Balance              $119.08   
22                                                                           
23  Online transfer from Sav ...XXXX Transaction#:...       50.0    169.08   
24                          Weee Inc. 510-573-4967 CA     -70.61     98.47   
25                        SBU-SAC Cafe Stony Brook NY     -12.89     85.58   
26  Online transfer from Sav ...XXXX Transaction#:...       26.7    112.28   
27  Online transfer from Student A ...      14.21    126.49   
28  Online transfer from Sav ...XXXX Transaction#:...       40.0    166.49   
29         SBU Sac Dunkin Donuts Stony Brook NY 11/09      -2.93    163.56   
30                           Aldi 73013 Lake Grove NY     -26.77    136.79   
31                Fedex32928309 800-463-3339 TN 11/13     -31.19     105.6   
32           Amazon.com*B86X05ZQ0 Amzn.com/bill 11/11     -31.19     74.41   
33           Amazon.com*BT3R00QI2 Amzn.com/bill 11/11       -5.4     69.01   
34  Online tra

In [7]:
final_df['description'] = final_df['description'].str.replace(r'Web ID[:#]?\s*(\d{10}$)','', regex=True)
final_df

date  \
0                             
1                     09/26   
2                     09/29   
3                             
4                     09/29   
5                             
6                     09/29   
7                     10/01   
8                     10/01   
9                     10/06   
10                    10/07   
11                    10/07   
12                    10/09   
13                    10/20   
14                    10/27   
15                    10/27   
16                            
17                    10/27   
18                            
19                    10/27   
20                    10/27   
21                            
22  *end*transaction detail   
23                    10/28   
24                    10/29   
25                    10/30   
26                    10/31   
27                    10/31   
28                    10/31   
29                    11/09   
30                    11/09   
31                    11/12   
32                    11/12   
33                    11/12   
34                    11/15   
35                    11/15   
36                    11/15   
37                    11/15   
38                    11/15   
39                    11/15   
40                    11/15   
41                    11/15   
42                    11/15   
43                    11/15   

                                          description     amount   balance  \
0                                   Beginning Balance              $192.67   
1   Card Purchase           09/25 Weee Inc. 510-57...     -44.89    147.78   
2   Card Purchase With Pin  09/27 Burlington Store...      -8.37    139.41   
3                                                5834                        
4   Card Purchase With Pin  09/27 Trader Joe S #55...     -10.96    128.45   
5                                                5834                        
6   Card Purchase With Pin  09/27 Aldi 73013 Lake ...      -6.42    122.03   
7   Zelle Payment From Friend A
8   Online Transfer To  Sav ...XXXX Transaction#: ...     -25.00    124.48   
9   Card Purchase With Pin  10/05 Wal-Mart #2286 C...      -5.36    119.12   
10  Online Transfer From Sav ...XXXX Transaction#:...   1,191.00  1,310.12   
11  Zelle Payment From Friend A
12       Sbufeesdeposit   Purchase   168304            -1,191.00    121.87   
13  Zelle Payment To Friend A Sbu 266477...     -10.25    111.62   
14  Online Transfer From Sav ...XXXX Transaction#:...      50.00    161.62   
15  Card Purchase           10/24 Sbu Sac Dunkin D...      -4.45    157.17   
16                                               5834                        
17  Card Purchase With Pin  10/25 Burlington Store...      -7.32    149.85   
18                                               5834                        
19  Card Purchase With Pin  10/25 Aldi 73013 Lake ...     -17.24    132.61   
20  Card Purchase With Pin  10/26 Wal-Mart #2286 C...     -13.53    119.08   
21                                     Ending Balance              $119.08   
22                                                                           
23  Online transfer from Sav ...XXXX Transaction#:...       50.0    169.08   
24                          Weee Inc. 510-573-4967 CA     -70.61     98.47   
25                        SBU-SAC Cafe Stony Brook NY     -12.89     85.58   
26  Online transfer from Sav ...XXXX Transaction#:...       26.7    112.28   
27  Online transfer from Student A ...      14.21    126.49   
28  Online transfer from Sav ...XXXX Transaction#:...       40.0    166.49   
29         SBU Sac Dunkin Donuts Stony Brook NY 11/09      -2.93    163.56   
30                           Aldi 73013 Lake Grove NY     -26.77    136.79   
31                Fedex32928309 800-463-3339 TN 11/13     -31.19     105.6   
32           Amazon.com*B86X05ZQ0 Amzn.com/bill 11/11     -31.19     74.41   
33           Amazon.com*BT3R00QI2 Amzn.com/bill 11/11       -5.4     69.01   
34  Online tra

In [8]:
final_df['description'] = final_df['description'].str.replace(r'(?:Transaction#[:]?\s*)?(\d{11}$)','', regex=True)
final_df

date  \
0                             
1                     09/26   
2                     09/29   
3                             
4                     09/29   
5                             
6                     09/29   
7                     10/01   
8                     10/01   
9                     10/06   
10                    10/07   
11                    10/07   
12                    10/09   
13                    10/20   
14                    10/27   
15                    10/27   
16                            
17                    10/27   
18                            
19                    10/27   
20                    10/27   
21                            
22  *end*transaction detail   
23                    10/28   
24                    10/29   
25                    10/30   
26                    10/31   
27                    10/31   
28                    10/31   
29                    11/09   
30                    11/09   
31                    11/12   
32                    11/12   
33                    11/12   
34                    11/15   
35                    11/15   
36                    11/15   
37                    11/15   
38                    11/15   
39                    11/15   
40                    11/15   
41                    11/15   
42                    11/15   
43                    11/15   

                                          description     amount   balance  \
0                                   Beginning Balance              $192.67   
1   Card Purchase           09/25 Weee Inc. 510-57...     -44.89    147.78   
2   Card Purchase With Pin  09/27 Burlington Store...      -8.37    139.41   
3                                                5834                        
4   Card Purchase With Pin  09/27 Trader Joe S #55...     -10.96    128.45   
5                                                5834                        
6   Card Purchase With Pin  09/27 Aldi 73013 Lake ...      -6.42    122.03   
7        Zelle Payment From Friend A
8                    Online Transfer To  Sav ...XXXX      -25.00    124.48   
9   Card Purchase With Pin  10/05 Wal-Mart #2286 C...      -5.36    119.12   
10                  Online Transfer From Sav ...XXXX    1,191.00  1,310.12   
11             Zelle Payment From Friend A
12       Sbufeesdeposit   Purchase   168304            -1,191.00    121.87   
13           Zelle Payment To Friend A Sbu      -10.25    111.62   
14                  Online Transfer From Sav ...XXXX       50.00    161.62   
15  Card Purchase           10/24 Sbu Sac Dunkin D...      -4.45    157.17   
16                                               5834                        
17  Card Purchase With Pin  10/25 Burlington Store...      -7.32    149.85   
18                                               5834                        
19  Card Purchase With Pin  10/25 Aldi 73013 Lake ...     -17.24    132.61   
20  Card Purchase With Pin  10/26 Wal-Mart #2286 C...     -13.53    119.08   
21                                     Ending Balance              $119.08   
22                                                                           
23                  Online transfer from Sav ...XXXX        50.0    169.08   
24                          Weee Inc. 510-573-4967 CA     -70.61     98.47   
25                        SBU-SAC Cafe Stony Brook NY     -12.89     85.58   
26                  Online transfer from Sav ...XXXX        26.7    112.28   
27     Online transfer from Student A       14.21    126.49   
28                  Online transfer from Sav ...XXXX        40.0    166.49   
29         SBU Sac Dunkin Donuts Stony Brook NY 11/09      -2.93    163.56   
30                           Aldi 73013 Lake Grove NY     -26.77    136.79   
31                Fedex32928309 800-463-3339 TN 11/13     -31.19     105.6   
32           Amazon.com*B86X05ZQ0 Amzn.com/bill 11/11     -31.19     74.41   
33           Amazon.com*BT3R00QI2 Amzn.com/bill 11/11       -5.4     69.01  

In [9]:
final_df[['Web ID', 'Transaction ID']] = final_df[['Web ID', 'Transaction ID']].fillna('N/A')
final_df

date  \
0                             
1                     09/26   
2                     09/29   
3                             
4                     09/29   
5                             
6                     09/29   
7                     10/01   
8                     10/01   
9                     10/06   
10                    10/07   
11                    10/07   
12                    10/09   
13                    10/20   
14                    10/27   
15                    10/27   
16                            
17                    10/27   
18                            
19                    10/27   
20                    10/27   
21                            
22  *end*transaction detail   
23                    10/28   
24                    10/29   
25                    10/30   
26                    10/31   
27                    10/31   
28                    10/31   
29                    11/09   
30                    11/09   
31                    11/12   
32                    11/12   
33                    11/12   
34                    11/15   
35                    11/15   
36                    11/15   
37                    11/15   
38                    11/15   
39                    11/15   
40                    11/15   
41                    11/15   
42                    11/15   
43                    11/15   

                                          description     amount   balance  \
0                                   Beginning Balance              $192.67   
1   Card Purchase           09/25 Weee Inc. 510-57...     -44.89    147.78   
2   Card Purchase With Pin  09/27 Burlington Store...      -8.37    139.41   
3                                                5834                        
4   Card Purchase With Pin  09/27 Trader Joe S #55...     -10.96    128.45   
5                                                5834                        
6   Card Purchase With Pin  09/27 Aldi 73013 Lake ...      -6.42    122.03   
7        Zelle Payment From Friend A
8                    Online Transfer To  Sav ...XXXX      -25.00    124.48   
9   Card Purchase With Pin  10/05 Wal-Mart #2286 C...      -5.36    119.12   
10                  Online Transfer From Sav ...XXXX    1,191.00  1,310.12   
11             Zelle Payment From Friend A
12       Sbufeesdeposit   Purchase   168304            -1,191.00    121.87   
13           Zelle Payment To Friend A Sbu      -10.25    111.62   
14                  Online Transfer From Sav ...XXXX       50.00    161.62   
15  Card Purchase           10/24 Sbu Sac Dunkin D...      -4.45    157.17   
16                                               5834                        
17  Card Purchase With Pin  10/25 Burlington Store...      -7.32    149.85   
18                                               5834                        
19  Card Purchase With Pin  10/25 Aldi 73013 Lake ...     -17.24    132.61   
20  Card Purchase With Pin  10/26 Wal-Mart #2286 C...     -13.53    119.08   
21                                     Ending Balance              $119.08   
22                                                                           
23                  Online transfer from Sav ...XXXX        50.0    169.08   
24                          Weee Inc. 510-573-4967 CA     -70.61     98.47   
25                        SBU-SAC Cafe Stony Brook NY     -12.89     85.58   
26                  Online transfer from Sav ...XXXX        26.7    112.28   
27     Online transfer from Student A       14.21    126.49   
28                  Online transfer from Sav ...XXXX        40.0    166.49   
29         SBU Sac Dunkin Donuts Stony Brook NY 11/09      -2.93    163.56   
30                           Aldi 73013 Lake Grove NY     -26.77    136.79   
31                Fedex32928309 800-463-3339 TN 11/13     -31.19     105.6   
32           Amazon.com*B86X05ZQ0 Amzn.com/bill 11/11     -31.19     74.41   
33           Amazon.com*BT3R00QI2 Amzn.com/bill 11/11       -5.4     69.01  

In [10]:
import numpy as np
final_df['date'] = final_df['date'].replace('', np.nan)
final_df = final_df.dropna(subset=['date'])

final_df

date  \
1                     09/26   
2                     09/29   
4                     09/29   
6                     09/29   
7                     10/01   
8                     10/01   
9                     10/06   
10                    10/07   
11                    10/07   
12                    10/09   
13                    10/20   
14                    10/27   
15                    10/27   
17                    10/27   
19                    10/27   
20                    10/27   
22  *end*transaction detail   
23                    10/28   
24                    10/29   
25                    10/30   
26                    10/31   
27                    10/31   
28                    10/31   
29                    11/09   
30                    11/09   
31                    11/12   
32                    11/12   
33                    11/12   
34                    11/15   
35                    11/15   
36                    11/15   
37                    11/15   
38                    11/15   
39                    11/15   
40                    11/15   
41                    11/15   
42                    11/15   
43                    11/15   

                                          description     amount   balance  \
1   Card Purchase           09/25 Weee Inc. 510-57...     -44.89    147.78   
2   Card Purchase With Pin  09/27 Burlington Store...      -8.37    139.41   
4   Card Purchase With Pin  09/27 Trader Joe S #55...     -10.96    128.45   
6   Card Purchase With Pin  09/27 Aldi 73013 Lake ...      -6.42    122.03   
7        Zelle Payment From Friend A
8                    Online Transfer To  Sav ...XXXX      -25.00    124.48   
9   Card Purchase With Pin  10/05 Wal-Mart #2286 C...      -5.36    119.12   
10                  Online Transfer From Sav ...XXXX    1,191.00  1,310.12   
11             Zelle Payment From Friend A
12       Sbufeesdeposit   Purchase   168304            -1,191.00    121.87   
13           Zelle Payment To Friend A Sbu      -10.25    111.62   
14                  Online Transfer From Sav ...XXXX       50.00    161.62   
15  Card Purchase           10/24 Sbu Sac Dunkin D...      -4.45    157.17   
17  Card Purchase With Pin  10/25 Burlington Store...      -7.32    149.85   
19  Card Purchase With Pin  10/25 Aldi 73013 Lake ...     -17.24    132.61   
20  Card Purchase With Pin  10/26 Wal-Mart #2286 C...     -13.53    119.08   
22                                                                           
23                  Online transfer from Sav ...XXXX        50.0    169.08   
24                          Weee Inc. 510-573-4967 CA     -70.61     98.47   
25                        SBU-SAC Cafe Stony Brook NY     -12.89     85.58   
26                  Online transfer from Sav ...XXXX        26.7    112.28   
27     Online transfer from Student A       14.21    126.49   
28                  Online transfer from Sav ...XXXX        40.0    166.49   
29         SBU Sac Dunkin Donuts Stony Brook NY 11/09      -2.93    163.56   
30                           Aldi 73013 Lake Grove NY     -26.77    136.79   
31                Fedex32928309 800-463-3339 TN 11/13     -31.19     105.6   
32           Amazon.com*B86X05ZQ0 Amzn.com/bill 11/11     -31.19     74.41   
33           Amazon.com*BT3R00QI2 Amzn.com/bill 11/11       -5.4     69.01   
34                  Online transfer from Sav ...XXXX      1191.0   1260.01   
35                  Online transfer from Sav ...XXXX        80.0   1340.01   
36  Card purchase MTA*LIRR Etix Ticket 718-217-547...     -15.25   1324.76   
37          Card purchase SQ *Aliou Dieye New York NY      -22.2   1302.56   
38           Card purchase Dunkin #358307 New York NY      -11.6   1290.96   
39            Card purchase MTA*NYC PayGo New York NY       -2.9   1288.06   
40           Card purchase SQ *Mery Guano New York NY      -8.32   1279.74   
41      Card purchase Orozcos Gas PNY LLC Brooklyn NY      -5.44    1274.3   
42        Card purchase SQ *

In [11]:
final_df['date'] = final_df['date'].str.extract(r'(\d{2}/\d{2})')
final_df = final_df.dropna(subset=['date'])
final_df

C:\Users\vdsha\AppData\Local\Temp\ipykernel_35872\3477473785.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['date'] = final_df['date'].str.extract(r'(\d{2}/\d{2})')


date                                        description     amount  \
1   09/26  Card Purchase           09/25 Weee Inc. 510-57...     -44.89   
2   09/29  Card Purchase With Pin  09/27 Burlington Store...      -8.37   
4   09/29  Card Purchase With Pin  09/27 Trader Joe S #55...     -10.96   
6   09/29  Card Purchase With Pin  09/27 Aldi 73013 Lake ...      -6.42   
7   10/01       Zelle Payment From Friend A
8   10/01                   Online Transfer To  Sav ...XXXX      -25.00   
9   10/06  Card Purchase With Pin  10/05 Wal-Mart #2286 C...      -5.36   
10  10/07                  Online Transfer From Sav ...XXXX    1,191.00   
11  10/07             Zelle Payment From Friend A
12  10/09       Sbufeesdeposit   Purchase   168304            -1,191.00   
13  10/20           Zelle Payment To Friend A Sbu      -10.25   
14  10/27                  Online Transfer From Sav ...XXXX       50.00   
15  10/27  Card Purchase           10/24 Sbu Sac Dunkin D...      -4.45   
17  10/27  Card Purchase With Pin  10/25 Burlington Store...      -7.32   
19  10/27  Card Purchase With Pin  10/25 Aldi 73013 Lake ...     -17.24   
20  10/27  Card Purchase With Pin  10/26 Wal-Mart #2286 C...     -13.53   
23  10/28                  Online transfer from Sav ...XXXX        50.0   
24  10/29                          Weee Inc. 510-573-4967 CA     -70.61   
25  10/30                        SBU-SAC Cafe Stony Brook NY     -12.89   
26  10/31                  Online transfer from Sav ...XXXX        26.7   
27  10/31     Online transfer from Student A       14.21   
28  10/31                  Online transfer from Sav ...XXXX        40.0   
29  11/09         SBU Sac Dunkin Donuts Stony Brook NY 11/09      -2.93   
30  11/09                           Aldi 73013 Lake Grove NY     -26.77   
31  11/12                Fedex32928309 800-463-3339 TN 11/13     -31.19   
32  11/12           Amazon.com*B86X05ZQ0 Amzn.com/bill 11/11     -31.19   
33  11/12           Amazon.com*BT3R00QI2 Amzn.com/bill 11/11       -5.4   
34  11/15                  Online transfer from Sav ...XXXX      1191.0   
35  11/15                  Online transfer from Sav ...XXXX        80.0   
36  11/15  Card purchase MTA*LIRR Etix Ticket 718-217-547...     -15.25   
37  11/15          Card purchase SQ *Aliou Dieye New York NY      -22.2   
38  11/15           Card purchase Dunkin #358307 New York NY      -11.6   
39  11/15            Card purchase MTA*NYC PayGo New York NY       -2.9   
40  11/15           Card purchase SQ *Mery Guano New York NY      -8.32   
41  11/15      Card purchase Orozcos Gas PNY LLC Brooklyn NY      -5.44   
42  11/15        Card purchase SQ *Aguas y Sodas Brooklyn NY       -1.0   
43  11/15                 Card purchase 7-Eleven Brooklyn NY       -1.0   

     balance      Web ID Transaction ID  
1     147.78         N/A            N/A  
2     139.41         N/A            N/A  
4     128.45         N/A            N/A  
6     122.03         N/A            N/A  
7     149.48         N/A    12345678901  
8     124.48         N/A    12345678901  
9     119.12         N/A            N/A  
10  1,310.12         N/A    12345678901  
11  1,312.87         N/A    12345678901  
12    121.87  9354449001            N/A  
13    111.62         N/A    12345678901  
14    161.62         N/A    12345678901  
15    157.17         N/A            N/A  
17    149.85         N/A            N/A  
19    132.61         N/A            N/A  
20    119.08         N/A            N/A  
23    169.08         N/A    12345678901  
24     98.47         N/A            N/A  
25     85.58         N/A            N/A  
26    112.28         N/A    12345678901  
27    126.49         N/A    12345678901  
28    166.49         N/A    12345678901  
29    163.56         N/A            N/A  
30    136.79         N/A            N/A  
31     105.6         N/A            N/A  
32     74.41         N/A            N/A  
33     69.01         N/A            N/A  
34   1260.01         N/A    12345678901  
35   1340.01     

In [12]:
final_df.columns = ['Date', 'Description', 'Amount', 'Balance', 'Web ID', 'Transaction ID']
final_df

Date                                        Description     Amount  \
1   09/26  Card Purchase           09/25 Weee Inc. 510-57...     -44.89   
2   09/29  Card Purchase With Pin  09/27 Burlington Store...      -8.37   
4   09/29  Card Purchase With Pin  09/27 Trader Joe S #55...     -10.96   
6   09/29  Card Purchase With Pin  09/27 Aldi 73013 Lake ...      -6.42   
7   10/01       Zelle Payment From Friend A
8   10/01                   Online Transfer To  Sav ...XXXX      -25.00   
9   10/06  Card Purchase With Pin  10/05 Wal-Mart #2286 C...      -5.36   
10  10/07                  Online Transfer From Sav ...XXXX    1,191.00   
11  10/07             Zelle Payment From Friend A
12  10/09       Sbufeesdeposit   Purchase   168304            -1,191.00   
13  10/20           Zelle Payment To Friend A Sbu      -10.25   
14  10/27                  Online Transfer From Sav ...XXXX       50.00   
15  10/27  Card Purchase           10/24 Sbu Sac Dunkin D...      -4.45   
17  10/27  Card Purchase With Pin  10/25 Burlington Store...      -7.32   
19  10/27  Card Purchase With Pin  10/25 Aldi 73013 Lake ...     -17.24   
20  10/27  Card Purchase With Pin  10/26 Wal-Mart #2286 C...     -13.53   
23  10/28                  Online transfer from Sav ...XXXX        50.0   
24  10/29                          Weee Inc. 510-573-4967 CA     -70.61   
25  10/30                        SBU-SAC Cafe Stony Brook NY     -12.89   
26  10/31                  Online transfer from Sav ...XXXX        26.7   
27  10/31     Online transfer from Student A       14.21   
28  10/31                  Online transfer from Sav ...XXXX        40.0   
29  11/09         SBU Sac Dunkin Donuts Stony Brook NY 11/09      -2.93   
30  11/09                           Aldi 73013 Lake Grove NY     -26.77   
31  11/12                Fedex32928309 800-463-3339 TN 11/13     -31.19   
32  11/12           Amazon.com*B86X05ZQ0 Amzn.com/bill 11/11     -31.19   
33  11/12           Amazon.com*BT3R00QI2 Amzn.com/bill 11/11       -5.4   
34  11/15                  Online transfer from Sav ...XXXX      1191.0   
35  11/15                  Online transfer from Sav ...XXXX        80.0   
36  11/15  Card purchase MTA*LIRR Etix Ticket 718-217-547...     -15.25   
37  11/15          Card purchase SQ *Aliou Dieye New York NY      -22.2   
38  11/15           Card purchase Dunkin #358307 New York NY      -11.6   
39  11/15            Card purchase MTA*NYC PayGo New York NY       -2.9   
40  11/15           Card purchase SQ *Mery Guano New York NY      -8.32   
41  11/15      Card purchase Orozcos Gas PNY LLC Brooklyn NY      -5.44   
42  11/15        Card purchase SQ *Aguas y Sodas Brooklyn NY       -1.0   
43  11/15                 Card purchase 7-Eleven Brooklyn NY       -1.0   

     Balance      Web ID Transaction ID  
1     147.78         N/A            N/A  
2     139.41         N/A            N/A  
4     128.45         N/A            N/A  
6     122.03         N/A            N/A  
7     149.48         N/A    12345678901  
8     124.48         N/A    12345678901  
9     119.12         N/A            N/A  
10  1,310.12         N/A    12345678901  
11  1,312.87         N/A    12345678901  
12    121.87  9354449001            N/A  
13    111.62         N/A    12345678901  
14    161.62         N/A    12345678901  
15    157.17         N/A            N/A  
17    149.85         N/A            N/A  
19    132.61         N/A            N/A  
20    119.08         N/A            N/A  
23    169.08         N/A    12345678901  
24     98.47         N/A            N/A  
25     85.58         N/A            N/A  
26    112.28         N/A    12345678901  
27    126.49         N/A    12345678901  
28    166.49         N/A    12345678901  
29    163.56         N/A            N/A  
30    136.79         N/A            N/A  
31     105.6         N/A            N/A  
32     74.41         N/A            N/A  
33     69.01         N/A            N/A  
34   1260.01         N/A    12345678901  
35   1340.01     

In [13]:
final_df.dtypes

Date              object
Description       object
Amount            object
Balance           object
Web ID            object
Transaction ID    object
dtype: object

In [14]:
final_df[['Amount', 'Balance']] = final_df[['Amount', 'Balance']].replace(',','',regex=True)
final_df.reset_index(drop=True, inplace=True)
final_df

C:\Users\vdsha\AppData\Local\Temp\ipykernel_35872\19883864.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[['Amount', 'Balance']] = final_df[['Amount', 'Balance']].replace(',','',regex=True)


Date                                        Description    Amount  \
0   09/26  Card Purchase           09/25 Weee Inc. 510-57...    -44.89   
1   09/29  Card Purchase With Pin  09/27 Burlington Store...     -8.37   
2   09/29  Card Purchase With Pin  09/27 Trader Joe S #55...    -10.96   
3   09/29  Card Purchase With Pin  09/27 Aldi 73013 Lake ...     -6.42   
4   10/01       Zelle Payment From Friend A
5   10/01                   Online Transfer To  Sav ...XXXX     -25.00   
6   10/06  Card Purchase With Pin  10/05 Wal-Mart #2286 C...     -5.36   
7   10/07                  Online Transfer From Sav ...XXXX    1191.00   
8   10/07             Zelle Payment From Friend A
9   10/09       Sbufeesdeposit   Purchase   168304            -1191.00   
10  10/20           Zelle Payment To Friend A Sbu     -10.25   
11  10/27                  Online Transfer From Sav ...XXXX      50.00   
12  10/27  Card Purchase           10/24 Sbu Sac Dunkin D...     -4.45   
13  10/27  Card Purchase With Pin  10/25 Burlington Store...     -7.32   
14  10/27  Card Purchase With Pin  10/25 Aldi 73013 Lake ...    -17.24   
15  10/27  Card Purchase With Pin  10/26 Wal-Mart #2286 C...    -13.53   
16  10/28                  Online transfer from Sav ...XXXX       50.0   
17  10/29                          Weee Inc. 510-573-4967 CA    -70.61   
18  10/30                        SBU-SAC Cafe Stony Brook NY    -12.89   
19  10/31                  Online transfer from Sav ...XXXX       26.7   
20  10/31     Online transfer from Student A      14.21   
21  10/31                  Online transfer from Sav ...XXXX       40.0   
22  11/09         SBU Sac Dunkin Donuts Stony Brook NY 11/09     -2.93   
23  11/09                           Aldi 73013 Lake Grove NY    -26.77   
24  11/12                Fedex32928309 800-463-3339 TN 11/13    -31.19   
25  11/12           Amazon.com*B86X05ZQ0 Amzn.com/bill 11/11    -31.19   
26  11/12           Amazon.com*BT3R00QI2 Amzn.com/bill 11/11      -5.4   
27  11/15                  Online transfer from Sav ...XXXX     1191.0   
28  11/15                  Online transfer from Sav ...XXXX       80.0   
29  11/15  Card purchase MTA*LIRR Etix Ticket 718-217-547...    -15.25   
30  11/15          Card purchase SQ *Aliou Dieye New York NY     -22.2   
31  11/15           Card purchase Dunkin #358307 New York NY     -11.6   
32  11/15            Card purchase MTA*NYC PayGo New York NY      -2.9   
33  11/15           Card purchase SQ *Mery Guano New York NY     -8.32   
34  11/15      Card purchase Orozcos Gas PNY LLC Brooklyn NY     -5.44   
35  11/15        Card purchase SQ *Aguas y Sodas Brooklyn NY      -1.0   
36  11/15                 Card purchase 7-Eleven Brooklyn NY      -1.0   

    Balance      Web ID Transaction ID  
0    147.78         N/A            N/A  
1    139.41         N/A            N/A  
2    128.45         N/A            N/A  
3    122.03         N/A            N/A  
4    149.48         N/A    12345678901  
5    124.48         N/A    12345678901  
6    119.12         N/A            N/A  
7   1310.12         N/A    12345678901  
8   1312.87         N/A    12345678901  
9    121.87  9354449001            N/A  
10   111.62         N/A    12345678901  
11   161.62         N/A    12345678901  
12   157.17         N/A            N/A  
13   149.85         N/A            N/A  
14   132.61         N/A            N/A  
15   119.08         N/A            N/A  
16   169.08         N/A    12345678901  
17    98.47         N/A            N/A  
18    85.58         N/A            N/A  
19   112.28         N/A    12345678901  
20   126.49         N/A    12345678901  
21   166.49         N/A    12345678901  
22   163.56         N/A            N/A  
23   136.79         N/A            N/A  
24    105.6         N/A            N/A  
25    74.41         N/A            N/A  
26    69.01         N/A            N/A  
27  1260.01         N/A    12345678901  
28  1340.01         N/A    12345678901  
29  1324.76         N/A            N/A  


In [15]:
final_df[['Amount', 'Balance']] = final_df[['Amount', 'Balance']].astype(float)
final_df = final_df.copy()

final_df['Date'] = final_df['Date'].astype(str).str.strip()
final_df['Date'] = final_df['Date'] + '/2025'
final_df['Date'] = pd.to_datetime(final_df['Date'], format='%m/%d/%Y')

final_df['Date']
# final_df.dtypes

C:\Users\vdsha\AppData\Local\Temp\ipykernel_35872\431338340.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[['Amount', 'Balance']] = final_df[['Amount', 'Balance']].astype(float)


0    2025-09-26
1    2025-09-29
2    2025-09-29
3    2025-09-29
4    2025-10-01
5    2025-10-01
6    2025-10-06
7    2025-10-07
8    2025-10-07
9    2025-10-09
10   2025-10-20
11   2025-10-27
12   2025-10-27
13   2025-10-27
14   2025-10-27
15   2025-10-27
16   2025-10-28
17   2025-10-29
18   2025-10-30
19   2025-10-31
20   2025-10-31
21   2025-10-31
22   2025-11-09
23   2025-11-09
24   2025-11-12
25   2025-11-12
26   2025-11-12
27   2025-11-15
28   2025-11-15
29   2025-11-15
30   2025-11-15
31   2025-11-15
32   2025-11-15
33   2025-11-15
34   2025-11-15
35   2025-11-15
36   2025-11-15
Name: Date, dtype: datetime64[ns]

In [16]:
final_df

Date                                        Description   Amount  \
0  2025-09-26  Card Purchase           09/25 Weee Inc. 510-57...   -44.89   
1  2025-09-29  Card Purchase With Pin  09/27 Burlington Store...    -8.37   
2  2025-09-29  Card Purchase With Pin  09/27 Trader Joe S #55...   -10.96   
3  2025-09-29  Card Purchase With Pin  09/27 Aldi 73013 Lake ...    -6.42   
4  2025-10-01       Zelle Payment From Friend A
5  2025-10-01                   Online Transfer To  Sav ...XXXX    -25.00   
6  2025-10-06  Card Purchase With Pin  10/05 Wal-Mart #2286 C...    -5.36   
7  2025-10-07                  Online Transfer From Sav ...XXXX   1191.00   
8  2025-10-07             Zelle Payment From Friend A
9  2025-10-09       Sbufeesdeposit   Purchase   168304           -1191.00   
10 2025-10-20           Zelle Payment To Friend A Sbu    -10.25   
11 2025-10-27                  Online Transfer From Sav ...XXXX     50.00   
12 2025-10-27  Card Purchase           10/24 Sbu Sac Dunkin D...    -4.45   
13 2025-10-27  Card Purchase With Pin  10/25 Burlington Store...    -7.32   
14 2025-10-27  Card Purchase With Pin  10/25 Aldi 73013 Lake ...   -17.24   
15 2025-10-27  Card Purchase With Pin  10/26 Wal-Mart #2286 C...   -13.53   
16 2025-10-28                  Online transfer from Sav ...XXXX     50.00   
17 2025-10-29                          Weee Inc. 510-573-4967 CA   -70.61   
18 2025-10-30                        SBU-SAC Cafe Stony Brook NY   -12.89   
19 2025-10-31                  Online transfer from Sav ...XXXX     26.70   
20 2025-10-31     Online transfer from Student A     14.21   
21 2025-10-31                  Online transfer from Sav ...XXXX     40.00   
22 2025-11-09         SBU Sac Dunkin Donuts Stony Brook NY 11/09    -2.93   
23 2025-11-09                           Aldi 73013 Lake Grove NY   -26.77   
24 2025-11-12                Fedex32928309 800-463-3339 TN 11/13   -31.19   
25 2025-11-12           Amazon.com*B86X05ZQ0 Amzn.com/bill 11/11   -31.19   
26 2025-11-12           Amazon.com*BT3R00QI2 Amzn.com/bill 11/11    -5.40   
27 2025-11-15                  Online transfer from Sav ...XXXX   1191.00   
28 2025-11-15                  Online transfer from Sav ...XXXX     80.00   
29 2025-11-15  Card purchase MTA*LIRR Etix Ticket 718-217-547...   -15.25   
30 2025-11-15          Card purchase SQ *Aliou Dieye New York NY   -22.20   
31 2025-11-15           Card purchase Dunkin #358307 New York NY   -11.60   
32 2025-11-15            Card purchase MTA*NYC PayGo New York NY    -2.90   
33 2025-11-15           Card purchase SQ *Mery Guano New York NY    -8.32   
34 2025-11-15      Card purchase Orozcos Gas PNY LLC Brooklyn NY    -5.44   
35 2025-11-15        Card purchase SQ *Aguas y Sodas Brooklyn NY    -1.00   
36 2025-11-15                 Card purchase 7-Eleven Brooklyn NY    -1.00   

    Balance      Web ID Transaction ID  
0    147.78         N/A            N/A  
1    139.41         N/A            N/A  
2    128.45         N/A            N/A  
3    122.03         N/A            N/A  
4    149.48         N/A    12345678901  
5    124.48         N/A    12345678901  
6    119.12         N/A            N/A  
7   1310.12         N/A    12345678901  
8   1312.87         N/A    12345678901  
9    121.87  9354449001            N/A  
10   111.62         N/A    12345678901  
11   161.62         N/A    12345678901  
12   157.17         N/A            N/A  
13   149.85         N/A            N/A  
14   132.61         N/A            N/A  
15   119.08         N/A            N/A  
16   169.08         N/A    12345678901  
17    98.47         N/A            N/A  
18    85.58         N/A            N/A  
19   112.28         N/A    12345678901  
20   126.49         N/A    12345678901  
21   166.49         N/A    12345678901  
22   163.56         N/A            N/A  
23   136.79         N/A            N/A  
24   105.60         N/A            N/A  
25    74.41         N/A            N/A  
26    69.01         N/A            N/A  
27  1260.01

In [17]:
final_df.dtypes

Date              datetime64[ns]
Description               object
Amount                   float64
Balance                  float64
Web ID                    object
Transaction ID            object
dtype: object

In [18]:
category_map = {
    'Groceries': ['weee', 'aldi', 'trader joe', 'walmart', 'wal-mart', 'cotsco'],
    'Rent': ['sbufeesdeposit'],
    'Tuition': ['sbufeesdeposit'],
    'Eating Out': ['dunkin', 'sbueats', 'chipotle'],
    'Shopping': ['burlington', 'target', 'macy\'s', 'amazon'],
    'Stationery': ['staples', 'office depot'],
    'Transfers/Settlements': ['zelle payment', 'online transfer'],
    'Income/Deposits': ['payment from', 'deposit'],
    'Fees/Charges': ['fee', 'charge', 'interest']
}


In [19]:
def categorize_transaction(desc, amount):
    if not isinstance(desc, str):
        return 'Other'

    desc = desc.lower()

    # 1. Special handling for 'sbufeesdeposit'
    if "sbufeesdeposit" in desc:
        if abs(amount) == 1191:         # your rent amount
            return "Rent"
        else:
            return "Tuition"
    for category, keyword in category_map.items():
        for k in keyword:
            if k in desc:
                return category
    return 'Other'

# final_df['Category'] = final_df['Description'].apply(categorize_transaction)
# final_df

final_df['Category'] = final_df.apply(
    lambda row: categorize_transaction(row['Description'], row['Amount']),
    axis=1
)


#.apply() is a method that applies a Python function to each element of a Series (column) or each row/column of a DataFrame.
# we cannot directly call our python fnc on the 'Description' column, bcuz it's a pandas Series(arrays) not strings.
# direct call can be made only if we pass a single row of the Description Series(column) :   categorize_transaction(final_df.loc[0, 'Description'])


REMEMBER:

**for category, keyword in category_map:    ------WRONGGGG**

You are iterating over the dictionary itself, not over .items().
**Iterating over a dict yields only keys, not key–value pairs.**
So Python tries to unpack one value (a key) into two variables (category, keyword) → and fails.

You must use:
**for category, keywords in category_map.items():**


In [21]:
final_df

Date                                        Description   Amount  \
0  2025-09-26  Card Purchase           09/25 Weee Inc. 510-57...   -44.89   
1  2025-09-29  Card Purchase With Pin  09/27 Burlington Store...    -8.37   
2  2025-09-29  Card Purchase With Pin  09/27 Trader Joe S #55...   -10.96   
3  2025-09-29  Card Purchase With Pin  09/27 Aldi 73013 Lake ...    -6.42   
4  2025-10-01       Zelle Payment From Friend A
5  2025-10-01                   Online Transfer To  Sav ...XXXX    -25.00   
6  2025-10-06  Card Purchase With Pin  10/05 Wal-Mart #2286 C...    -5.36   
7  2025-10-07                  Online Transfer From Sav ...XXXX   1191.00   
8  2025-10-07             Zelle Payment From Friend A
9  2025-10-09       Sbufeesdeposit   Purchase   168304           -1191.00   
10 2025-10-20           Zelle Payment To Friend A Sbu    -10.25   
11 2025-10-27                  Online Transfer From Sav ...XXXX     50.00   
12 2025-10-27  Card Purchase           10/24 Sbu Sac Dunkin D...    -4.45   
13 2025-10-27  Card Purchase With Pin  10/25 Burlington Store...    -7.32   
14 2025-10-27  Card Purchase With Pin  10/25 Aldi 73013 Lake ...   -17.24   
15 2025-10-27  Card Purchase With Pin  10/26 Wal-Mart #2286 C...   -13.53   
16 2025-10-28                  Online transfer from Sav ...XXXX     50.00   
17 2025-10-29                          Weee Inc. 510-573-4967 CA   -70.61   
18 2025-10-30                        SBU-SAC Cafe Stony Brook NY   -12.89   
19 2025-10-31                  Online transfer from Sav ...XXXX     26.70   
20 2025-10-31     Online transfer from Student A     14.21   
21 2025-10-31                  Online transfer from Sav ...XXXX     40.00   
22 2025-11-09         SBU Sac Dunkin Donuts Stony Brook NY 11/09    -2.93   
23 2025-11-09                           Aldi 73013 Lake Grove NY   -26.77   
24 2025-11-12                Fedex32928309 800-463-3339 TN 11/13   -31.19   
25 2025-11-12           Amazon.com*B86X05ZQ0 Amzn.com/bill 11/11   -31.19   
26 2025-11-12           Amazon.com*BT3R00QI2 Amzn.com/bill 11/11    -5.40   
27 2025-11-15                  Online transfer from Sav ...XXXX   1191.00   
28 2025-11-15                  Online transfer from Sav ...XXXX     80.00   
29 2025-11-15  Card purchase MTA*LIRR Etix Ticket 718-217-547...   -15.25   
30 2025-11-15          Card purchase SQ *Aliou Dieye New York NY   -22.20   
31 2025-11-15           Card purchase Dunkin #358307 New York NY   -11.60   
32 2025-11-15            Card purchase MTA*NYC PayGo New York NY    -2.90   
33 2025-11-15           Card purchase SQ *Mery Guano New York NY    -8.32   
34 2025-11-15      Card purchase Orozcos Gas PNY LLC Brooklyn NY    -5.44   
35 2025-11-15        Card purchase SQ *Aguas y Sodas Brooklyn NY    -1.00   
36 2025-11-15                 Card purchase 7-Eleven Brooklyn NY    -1.00   

    Balance      Web ID Transaction ID               Category  
0    147.78         N/A            N/A              Groceries  
1    139.41         N/A            N/A               Shopping  
2    128.45         N/A            N/A              Groceries  
3    122.03         N/A            N/A              Groceries  
4    149.48         N/A    12345678901  Transfers/Settlements  
5    124.48         N/A    12345678901  Transfers/Settlements  
6    119.12         N/A            N/A              Groceries  
7   1310.12         N/A    12345678901  Transfers/Settlements  
8   1312.87         N/A    12345678901  Transfers/Settlements  
9    121.87  9354449001            N/A                   Rent  
10   111.62         N/A    12345678901  Transfers/Settlements  
11   161.62         N/A    12345678901  Transfers/Settlements  
12   157.17         N/A            N/A             Eating Out  
13   149.85         N/A            N/A               Shopping  
14   132.61         N/A            N/A              Groceries  
15   119.08         N/A            N/A              Groceries  
16   169.08         N/A    12345678901  Transfers/Settlements  
17    9

In [42]:
merchant_list = ["Weee", "Aldi", "Trader Joe", "Walmart", "Wal-mart", "Burlington", "Staples", "Sbufeesdeposit", "Dunkin", "Amazon", "Zelle", "SBU-SAC Cafe", "7-Eleven", " MTA"]
def extract_merchant(desc):
    desc = desc.lower()
    for m in merchant_list:
        if m.lower() in desc:
            return m

    return "Unknown"

final_df['Merchant'] = final_df['Description'].apply(extract_merchant)
final_df

Date                                        Description   Amount  \
0  2025-09-26  Card Purchase           09/25 Weee Inc. 510-57...   -44.89   
1  2025-09-29  Card Purchase With Pin  09/27 Burlington Store...    -8.37   
2  2025-09-29  Card Purchase With Pin  09/27 Trader Joe S #55...   -10.96   
3  2025-09-29  Card Purchase With Pin  09/27 Aldi 73013 Lake ...    -6.42   
4  2025-10-01       Zelle Payment From Friend A
5  2025-10-01                   Online Transfer To  Sav ...XXXX    -25.00   
6  2025-10-06  Card Purchase With Pin  10/05 Wal-Mart #2286 C...    -5.36   
7  2025-10-07                  Online Transfer From Sav ...XXXX   1191.00   
8  2025-10-07             Zelle Payment From Friend A
9  2025-10-09       Sbufeesdeposit   Purchase   168304           -1191.00   
10 2025-10-20           Zelle Payment To Friend A Sbu    -10.25   
11 2025-10-27                  Online Transfer From Sav ...XXXX     50.00   
12 2025-10-27  Card Purchase           10/24 Sbu Sac Dunkin D...    -4.45   
13 2025-10-27  Card Purchase With Pin  10/25 Burlington Store...    -7.32   
14 2025-10-27  Card Purchase With Pin  10/25 Aldi 73013 Lake ...   -17.24   
15 2025-10-27  Card Purchase With Pin  10/26 Wal-Mart #2286 C...   -13.53   
16 2025-10-28                  Online transfer from Sav ...XXXX     50.00   
17 2025-10-29                          Weee Inc. 510-573-4967 CA   -70.61   
18 2025-10-30                        SBU-SAC Cafe Stony Brook NY   -12.89   
19 2025-10-31                  Online transfer from Sav ...XXXX     26.70   
20 2025-10-31     Online transfer from Student A     14.21   
21 2025-10-31                  Online transfer from Sav ...XXXX     40.00   
22 2025-11-09         SBU Sac Dunkin Donuts Stony Brook NY 11/09    -2.93   
23 2025-11-09                           Aldi 73013 Lake Grove NY   -26.77   
24 2025-11-12                Fedex32928309 800-463-3339 TN 11/13   -31.19   
25 2025-11-12           Amazon.com*B86X05ZQ0 Amzn.com/bill 11/11   -31.19   
26 2025-11-12           Amazon.com*BT3R00QI2 Amzn.com/bill 11/11    -5.40   
27 2025-11-15                  Online transfer from Sav ...XXXX   1191.00   
28 2025-11-15                  Online transfer from Sav ...XXXX     80.00   
29 2025-11-15  Card purchase MTA*LIRR Etix Ticket 718-217-547...   -15.25   
30 2025-11-15          Card purchase SQ *Aliou Dieye New York NY   -22.20   
31 2025-11-15           Card purchase Dunkin #358307 New York NY   -11.60   
32 2025-11-15            Card purchase MTA*NYC PayGo New York NY    -2.90   
33 2025-11-15           Card purchase SQ *Mery Guano New York NY    -8.32   
34 2025-11-15      Card purchase Orozcos Gas PNY LLC Brooklyn NY    -5.44   
35 2025-11-15        Card purchase SQ *Aguas y Sodas Brooklyn NY    -1.00   
36 2025-11-15                 Card purchase 7-Eleven Brooklyn NY    -1.00   

    Balance      Web ID Transaction ID               Category        Merchant  
0    147.78         N/A            N/A              Groceries            Weee  
1    139.41         N/A            N/A               Shopping      Burlington  
2    128.45         N/A            N/A              Groceries      Trader Joe  
3    122.03         N/A            N/A              Groceries            Aldi  
4    149.48         N/A    12345678901  Transfers/Settlements           Zelle  
5    124.48         N/A    12345678901  Transfers/Settlements         Unknown  
6    119.12         N/A            N/A              Groceries        Wal-mart  
7   1310.12         N/A    12345678901  Transfers/Settlements         Unknown  
8   1312.87         N/A    12345678901  Transfers/Settlements           Zelle  
9    121.87  9354449001            N/A                   Rent  Sbufeesdeposit  
10   111.62         N/A    12345678901  Transfers/Settlements           Zelle  
11   161.62         N/A    12345678901  Transfers/Settlements         Unknown  
12   157.17         N/A            N/A             Eating Out          Dunkin  
13   149.85         N/A            N/A 

In [23]:
import psycopg2

In [24]:
import os
from dotenv import load_dotenv
import psycopg2

load_dotenv()  # loads variables from .env

conn = psycopg2.connect(
    host=os.getenv("DB_HOST", "localhost"),
    database=os.getenv("DB_NAME", "etl"),
    user=os.getenv("DB_USER", "postgres"),
    password=os.getenv("DB_PASSWORD")
)
cur = conn.cursor()

In [25]:
categories = final_df['Category'].unique()    #extracting unique categories intoa  single list
print(categories)

['Groceries' 'Shopping' 'Transfers/Settlements' 'Rent' 'Eating Out'
 'Other']


In [26]:
category_map={}
for cat in categories:
    cur.execute("""
        INSERT INTO categories(category_name)
        VALUES(%s)
        ON CONFLICT (category_name) DO NOTHING    
        RETURNING category_id;
    """,(cat,))                                   # If the category already exists, skip the insert.(useful when we have 2 or more months data insert--we dont wanna reate duplicate categories
                                                  # we never insert VALUES directly using string formatting, like f string
                                                  # always use parameterized queries via the vars argument. psycopg2(or python->postgres driver)  
                                                  # handles the safe substitution of values for you.
                                                  # (cat,)-->syntax nusar we're converting cat into a tuple
    result=cur.fetchone()                         # check if the insert returned a value(category_id returned after insert)
                                                  # cur.fetchone() returns the result of executed row as a tuple
    if result:
        category_map[cat]=result[0]               # if insert was successful, we create a mapping of cat name --> cat id
    else:
        cur.execute("""SELECT category_id FROM categories WHERE category_name=%s """,(cat,))
        category_map[cat]=cur.fetchone()[0]
conn.commit()

In [75]:
# conn.rollback()


In [28]:
print(category_map)

{'Groceries': 1, 'Shopping': 2, 'Transfers/Settlements': 3, 'Rent': 4, 'Eating Out': 5, 'Other': 12}


In [67]:
merchants=final_df['Merchant'].unique()
print(merchants)

['Weee' 'Burlington' 'Trader Joe' 'Aldi' 'Zelle' 'Unknown' 'Wal-mart'
 'Sbufeesdeposit' 'Dunkin' 'SBU-SAC Cafe' 'Amazon' ' MTA' '7-Eleven']


In [69]:
merchants_map={}

for m in merchants:
    cur.execute("""
        INSERT INTO Merchants (merchant_name)    
        VALUES(%s)
        ON CONFLICT (merchant_name) do nothing
        RETURNING merchant_id
    """,(m,))
    result=cur.fetchone()
    if result:
        merchants_map[m]=result[0]
    else:
        cur.execute("""SELECT merchant_id FROM merchants WHERE merchant_name=%s""",(m,))
        merchants_map[m]=cur.fetchone()[0]

conn.commit()

In [71]:
print(merchants_map)

{'Weee': 1, 'Burlington': 2, 'Trader Joe': 3, 'Aldi': 4, 'Zelle': 5, 'Unknown': 6, 'Wal-mart': 7, 'Sbufeesdeposit': 8, 'Dunkin': 9, 'SBU-SAC Cafe': 28, 'Amazon': 11, ' MTA': 30, '7-Eleven': 31}


In [32]:
# final_df.dtypes

In [77]:
account_id = 1  # or whatever ID was returned earlier

for _, row in final_df.iterrows():
    cur.execute("""
        INSERT INTO transactions (
            account_id, category_id, merchant_id,
            transaction_date, description, amount, balance,
            web_id, online_transaction_id
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
         ON CONFLICT ON CONSTRAINT unique_txn DO NOTHING;
    """, (
        account_id,
        category_map[row['Category']],
        merchants_map[row['Merchant']],
        row['Date'].date() if hasattr(row['Date'], 'date') else row['Date'],
        row['Description'],
        row['Amount'],
        row['Balance'],
        row['Web ID'],
        row['Transaction ID']
    ))

conn.commit()
cur.close()
conn.close()


In [44]:
final_df.to_parquet("final_df_cleaned.parquet", index=False)
# optional smaller file:
final_df.to_parquet("final_df_cleaned_snappy.parquet", index=False, compression='snappy')
# csv as backup:
final_df.to_csv("final_df_cleaned.csv", index=False)

